# Analyse et visualisation de données avec Python
## Combiner des DataFrame avec Pandas
Questions
* Peut-on travaillers avec plusieurs sources de données?
* Comment combiner les données de deux DataFrames?

Objectifs
* Combiner les données de plusieurs fichiers en utilisant `concat` et `merge`.
* Combiner deux DataFrames utilisant un identifiant commun.

## Charger nos données

In [ ]:
# Charger le module pandas
import pandas as pd

# Charger les données
surveys_df = pd.read_csv("../data/surveys.csv")

## Concaténer des DataFrames

In [ ]:
# Sélectionner les 10 premiers enregistrements
surveys_head10 = surveys_df.head(10)
surveys_head10

In [ ]:
# Sélectionner les 10 derniers enregistrements
surveys_tail10 = surveys_df.tail(10)
surveys_tail10

In [ ]:
# Concaténer les dataframes verticalement
vertical_stack = pd.concat([surveys_head10, surveys_tail10], axis=0)
vertical_stack

In [ ]:
# Réinitaliser l'index du dataframe
# L'option drop=True évite l'ajout d'une colonne avec l'ancien index
vertical_stack = vertical_stack.reset_index(drop=True)
vertical_stack

### Écrire le résultat dans un fichier CSV

In [ ]:
# Omettre l'index
vertical_stack.to_csv('surveys_sub.csv', index=False)

In [ ]:
# Charger le nouveau fichier CSV
surveys_sub = pd.read_csv('surveys_sub.csv')
surveys_sub

## Exercice - Concaténer des DataFrames
Dans `surveys_df`, sélectionnez individuellement les enregistrements des années 2001 et 2002. Concaténez les deux dataframes verticalement. Créez un *line-plot* montrant la moyenne des poids par type de sexe pour chaque année (note: le tableau de données final aura une ligne par année). Sauvegardez le tableau des moyennes dans un CSV et le recharger.

In [ ]:
# Obtenir les données pour chaque année
survey2001 = surveys_df[surveys_df['year'] == 2001]
survey2002 = surveys_df[surveys_df['year'] == 2002]

# Concaténer verticalement
survey_all = pd.concat([survey2001, survey2002], axis=0)

In [ ]:
# Calculer la moyenne par année et par sexe
weight_year = survey_all.groupby(['year', 'sex'])['weight'].mean().unstack()
weight_year

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Créer le graphique
weight_year.plot(kind="bar")

plt.tight_layout()  # Pour améliorer le graphique final

In [ ]:
# Écrire dans un fichier - garder l'index "year" cette fois-ci
weight_year.to_csv("weight_for_year.csv", index=True)

# Relire les données, fournir le nom de l'index
pd.read_csv("weight_for_year.csv", index_col='year')

## Joindre deux DataFrames

In [ ]:
# Importer un sous-ensemble des espèces pour cet exemple
species_sub = pd.read_csv('../data/speciesSubset.csv')
species_sub

### Identifier les clés de jonction

In [ ]:
surveys_head10.columns

In [ ]:
species_sub.columns

### Une intersection ou "inner join"

![Inner join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/inner-join.png)

In [ ]:
merged_inner = pd.merge(left=surveys_head10, right=species_sub,
                        left_on='species_id', right_on='species_id')
# Quelle est la taille de la jonction?
merged_inner.shape

In [ ]:
merged_inner

### Jonction de gauche

![Left join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/left-join.png)

In [ ]:
merged_left = pd.merge(left=surveys_head10, right=species_sub,
                       how='left', on='species_id')
# Quelle est la taille de la jonction?
merged_left.shape

In [ ]:
merged_left

In [ ]:
merged_left[ pd.isnull(merged_left['genus']) ]

### Les autres types de jonction
* `how='right'` : toutes les rangées du second DataFrame sont gardées
* `how='outer'` : équivalent d'une union, toutes les rangées sont gardées

## Exercice - Joindre toutes les données
`1`. Créez un nouveau DataFrame tel que tous les enregistrements de `surveys.csv` seront gardés, mais certains enregistrement seront joints aux informations correspondantes de `species.csv` (soit le fichier complet à charger).

In [ ]:
species_df = pd.read_csv("../data/species.csv")
merged_left = pd.merge(left=surveys_df, right=species_df, how='left', on='species_id')
merged_left.shape

`2`. Calculez et créez un graphique (*bar-plot*) montrant le nombre d'enregistrements (soit le nombre de `record_id`) par type de `taxa` pour chaque site (`plot_id`).

In [ ]:
taxa_site = merged_left.groupby(['plot_id', 'taxa'])['record_id'].count().unstack()
taxa_site.head()

In [ ]:
taxa_site.plot(kind='bar', stacked=True)
plt.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.1))

`3`. Calculez et créez un graphique (bar-plot) montrant le nombre de différents `taxa` par type de sexe pour chaque site (`plot_id`).

In [ ]:
# Nettoyage des données
merged_left.loc[merged_left['sex'].isnull(), 'sex'] = "F|M"
merged_left.loc[~merged_left['sex'].isin(["F", "F|M", "M"]), 'sex'] = "F|M"

In [ ]:
ntaxa_sex_site = merged_left.groupby(['plot_id',
                                      'sex'])['taxa'].nunique().reset_index(level=1)
ntaxa_sex_site.head()

In [ ]:
# Utiliser pivot_table() au lieu de unstack()
ntaxa_sex_site = ntaxa_sex_site.pivot_table(values='taxa', columns='sex',
                                            index=ntaxa_sex_site.index)
ntaxa_sex_site.head()

In [ ]:
ntaxa_sex_site.plot(kind="bar", stacked=True, legend=False)
plt.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.08),
           fontsize='small', frameon=False)